# PSI – Assistente Executivo Master  
## Análise de Dados – Desafio Técnico

Este notebook tem como objetivo demonstrar competências analíticas relacionadas à:

- Qualidade e tratamento de dados  
- Análise exploratória  
- Identificação de padrões e oportunidades de negócio  
- Geração de insights para apoio à tomada de decisão  

Os dados analisados referem-se a contratos de empréstimos, encargos financeiros, garantias e convenentes, sendo tratados e analisados no ambiente Databricks, em conformidade com boas práticas de governança de dados.


In [0]:
import pandas as pd

# Caminho dos arquivos no Databricks (Volumes - Unity Catalog)
base_path = "/Volumes/workspace/default/psi/"


In [0]:
df_convenentes = pd.read_csv(
    base_path + "psi_convenentes_2026-01-13_17-09-53.csv",
    sep=";"
)

df_contratos = pd.read_csv(
    base_path + "psi_emprestimo_contrato_2026-01-13_17-09-13.csv",
    sep=";"
)

df_encargos = pd.read_csv(
    base_path + "psi_emprestimo_contrato_encargo_financeiro_2026-01-13_17-11-41.csv",
    sep=";"
)

df_garantias = pd.read_csv(
    base_path + "psi_emprestimo_garantia_2026-01-13_17-09-33.csv.gz",
    sep=";",
    compression="gzip"
)

print("Dados carregados com sucesso")


Dados carregados com sucesso


In [0]:
# Padronização robusta de colunas (snake_case)
import re

def snake_case_columns(df):
    cols = df.columns.astype(str).str.strip()
    # camelCase -> snake_case
    cols = cols.str.replace(r'([a-z0-9])([A-Z])', r'\1_\2', regex=True)
    # caracteres não alfanuméricos viram underscore
    cols = cols.str.replace(r'[^0-9a-zA-Z_]', '_', regex=True)
    # espaços vários -> underscore
    cols = cols.str.replace(r'\s+', '_', regex=True)
    cols = cols.str.replace(r'_+', '_', regex=True)  # elimina underscores duplicados
    cols = cols.str.strip('_')
    cols = cols.str.lower()
    df.columns = cols
    return df

for df in [df_convenentes, df_contratos, df_encargos, df_garantias]:
    snake_case_columns(df)



In [0]:
# Conversão automática de colunas com 'date' ou 'data'
def convert_dates(df):
    date_cols = [c for c in df.columns if ('date' in c or 'data' in c)]
    for c in date_cols:
        try:
            df[c] = pd.to_datetime(df[c], errors='coerce')
        except Exception:
            pass

convert_dates(df_contratos)
convert_dates(df_garantias)
convert_dates(df_encargos)
convert_dates(df_convenentes)


In [0]:
# Conversão automática de colunas que provavelmente são numéricas
def convert_numerics(df):
    candidates = [c for c in df.columns if any(k in c for k in ["amount","value","valor","cet","rate","percent","ratio","quant"])]
    for c in candidates:
        df[c] = pd.to_numeric(df[c], errors='coerce')

convert_numerics(df_contratos)
convert_numerics(df_garantias)
convert_numerics(df_encargos)
convert_numerics(df_convenentes)


In [0]:
def qualidade_dados(df, nome):
    print(f"\n--- {nome} ---")
    display(df.info())
    display(df.head(3))
    print("Valores nulos:")
    display(df.isnull().sum())

qualidade_dados(df_convenentes, "Convenentes")
qualidade_dados(df_contratos, "Contratos")
qualidade_dados(df_encargos, "Encargos Financeiros")
qualidade_dados(df_garantias, "Garantias")



--- Convenentes ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   convenente       12 non-null     object 
 1   cpf_cnpj         12 non-null     object 
 2   codigo_situacao  12 non-null     int64  
 3   situacao         12 non-null     object 
 4   setor            12 non-null     object 
 5   prazo_inicial    12 non-null     int64  
 6   prazo_final      12 non-null     int64  
 7   faixa_b          12 non-null     float64
 8   faixa_c          12 non-null     float64
dtypes: float64(2), int64(3), object(4)
memory usage: 996.0+ bytes


convenente,cpf_cnpj,codigo_situacao,situacao,setor,prazo_inicial,prazo_final,faixa_b,faixa_c
814a81e9715216d6d590783669c926526a3b33170bcdc8e28847f7520e1c9c87804c9156f72c4a33feab7d62ded0c474288d16e2020cce18212c8b0a2af47f34,3d16df3e89406b3c088f2e6fd6536a67c8ae89075864e9564ae9162c7519407622e1b0348c3c8a3eb6f169d5e0926a8d5ecfb4a63a59734b670f9f48493cdfc1,1,ativo,público estadual,1,144,1.71,1.91
e026dd5a840991e12cfe8890556e6cd562767efebf6ee59f037e4ba8e0df75e72d6cce0505b33bf03a32531b927ad8722912ada60653fb25518c2cc271e5ae6f,2a07a3f5217cfa1360e816813ab06b73625ad9025d896e0b2c0e390eef12500d98ddcba97cfaed1cf9b08c5e4dc33b323fef8ea6eb0c72a68ddddcd12c2c1e2e,1,ativo,público municipal,1,144,1.83,2.0
108b46a534918c31dc2428a6427c6bef60da0d473ebaa7a39567d638ac5272a5d6681da38b321fa409076f0120deb1298ea797e862199ed9fcc39cd2b41d0c0b,314c763de3edb6f81ce3a542fbc996c0f01dd2fbf3acf345998bc75f7588c13512630f3c04ebdc28da6f77aad248ae918c7bb148c83097269f21b08b70a1f41d,1,ativo,público FEDERAL,1,96,1.63,1.68


Valores nulos:


convenente         0
cpf_cnpj           0
codigo_situacao    0
situacao           0
setor              0
prazo_inicial      0
prazo_final        0
faixa_b            0
faixa_c            0
dtype: int64


--- Contratos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                                  Non-Null Count   Dtype              
---  ------                                  --------------   -----              
 0   data_captura                            100000 non-null  datetime64[ns, UTC]
 1   consent_id                              100000 non-null  object             
 2   cpf_cnpj                                100000 non-null  object             
 3   nome_instituicao                        100000 non-null  object             
 4   contract_id                             100000 non-null  object             
 5   contract_number                         100000 non-null  object             
 6   ipoc_code                               100000 non-null  object             
 7   product_name                            100000 non-null  object             
 8   product_type                            100000

...

data_captura                  0
consent_id                    0
cpf_cnpj                      0
nome_instituicao              0
contract_id                   0
charge_type                   0
charge_additional_info    93942
charge_rate                8376
dtype: int64


--- Garantias ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   data_captura       100000 non-null  datetime64[ns, UTC]
 1   consent_id         100000 non-null  object             
 2   cpf_cnpj           100000 non-null  object             
 3   nome_instituicao   100000 non-null  object             
 4   contract_id        100000 non-null  object             
 5   currency           100000 non-null  object             
 6   warranty_type      100000 non-null  object             
 7   warranty_sub_type  99999 non-null   object             
 8   warranty_amount    100000 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(7)
memory usage: 6.9+ MB


data_captura,consent_id,cpf_cnpj,nome_instituicao,contract_id,currency,warranty_type,warranty_sub_type,warranty_amount
2025-12-26T19:09:23.157Z,urn:santander-pf:dados-92661eae-2e0c-4f9f-a45d-73d7a50bf67c,28c32c2d67604311f272b35035dfa5d794a97f5e56f2906a826ebb8220b575f454ec12e2e2a11cbe32b2fac3e92ac8b736677d334ef8eca03fdca6f1058ae67d,49f3bc0c1560fd92c6722e0fe91cf946ebe4c9cd5092aecaaafb2dc582f0dc2fa9a1dc5cb72463cf5e7018affb801e38fe3115eff973a033c1b4b4cf85ef6a95,6791c0046bf6af6870dc5c9cdf5ac5043a631bdec433a82ee2c896f87fb80ef9,BRL,SEGUROS_ASSEMELHADOS,FGTS_FUNDO_GARANTIA_TEMPO_SERVICO,96.92
2025-12-26T22:50:36.292Z,urn:nubank:6647cd8b-d828-38f7-bb6b-be4eac06f634,4287d70a37e01b85bb299ac7b50c01d4bc78ed7908bb3b2f8ee2303e85c37d8c1e3e60a23c6c19fd0980d1d670b7761d96c077149b0f0bb3b5f0514fda9ce12f,d3315d88d7e863945f0af67f9f0df831f4af159f174496e2d0c07465311c452f89c273839a530d685ff9e10df88f408cc896bbaad3d28182143f56a368fca19f,67863774-c168-404a-a3de-9a111bff1277,BRL,CAUCAO,APLICACOES_FINANCEIRAS_RENDA_FIXA,205.22
2025-12-26T22:22:24.750Z,urn:nubank:67bcbae4-8490-3b91-81a9-bac39626aa26,804ed2df975ab231285332ab81fb43ae350ab95a78800bf1001b05b2e3e3db9650c7fc7649c98f6d792b6ab9a0a41486b7fb839f9425f26f42ae882f0fbe94b9,d3315d88d7e863945f0af67f9f0df831f4af159f174496e2d0c07465311c452f89c273839a530d685ff9e10df88f408cc896bbaad3d28182143f56a368fca19f,679b991f-5c04-4785-a4b3-7a5a4e530518,BRL,CAUCAO,APLICACOES_FINANCEIRAS_RENDA_FIXA,909.62


Valores nulos:


data_captura         0
consent_id           0
cpf_cnpj             0
nome_instituicao     0
contract_id          0
currency             0
warranty_type        0
warranty_sub_type    1
warranty_amount      0
dtype: int64

In [0]:
# Distribuição dos valores dos contratos (usando nomes atualizados)
display(df_contratos["contract_amount"].describe())

# Instituições com maior volume de contratos (nome da coluna atualizada)
display(
    df_contratos["nome_instituicao"]
    .value_counts()
    .head(10)
)

# Tipos de encargos mais frequentes (colunas atualizadas)
display(
    df_encargos["charge_type"]
    .value_counts()
)

# Estatísticas das garantias (coluna atualizada)
display(df_garantias["warranty_amount"].describe())

# Tipos de garantia
display(
    df_garantias["warranty_type"]
    .value_counts()
)


count    100000.000000
mean       1728.821237
std        3831.858209
min           0.000000
25%         223.311025
50%         720.000000
75%        2000.000000
max      281607.900000
Name: contract_amount, dtype: float64

nome_instituicao
NU PAGAMENTOS S.A. - INSTITUICAO DE PAGAMENTO                      99112
BANCO BRADESCO SA                                                    386
MERCADO PAGO INSTITUICAO DE PAGAMENTO LTDA                           196
CONFEDERACAO DAS COOPERATIVAS DO SICREDI - CONFEDERACAO SICREDI      130
BCO SANTANDER (BRASIL) S.A.                                          126
BANCO BTG PACTUAL S.A.                                                29
BCO BMG S.A.                                                          11
BCO SOFISA S.A.                                                        5
BANCO INTER                                                            4
WILLBANK                                                               1
Name: count, dtype: int64

charge_type
JUROS_REMUNERATORIOS_POR_ATRASO    79655
JUROS_MORA_ATRASO                  10027
IOF_CONTRATACAO                     9497
MULTA_ATRASO_PAGAMENTO               421
SEM_ENCARGO                          224
IOF_POR_ATRASO                       119
OUTROS                                57
Name: count, dtype: int64

count    1.000000e+05
mean     3.713613e+03
std      4.010339e+05
min      0.000000e+00
25%      1.937100e+02
50%      3.026300e+02
75%      5.966125e+02
max      1.251612e+08
Name: warranty_amount, dtype: float64

warranty_type
CAUCAO                                   45103
OPERACOES_GARANTIDAS_PELO_GOVERNO        28827
SEGUROS_ASSEMELHADOS                     24260
GARANTIA_FIDEJUSSORIA                      703
CESSAO_DIREITOS_CREDITORIOS                346
OUTRAS_GARANTIAS_NAO_FIDEJUSSORIAS         329
OPERACOES_GARANTIDAS_OUTRAS_ENTIDADES      310
ALIENACAO_FIDUCIARIA                       110
HIPOTECA                                    11
PENHOR                                       1
Name: count, dtype: int64

In [0]:
# 1) Normalização e diagnóstico de chaves
df_contratos['contract_id'] = df_contratos['contract_id'].astype(str).str.strip()
df_garantias['contract_id'] = df_garantias['contract_id'].astype(str).str.strip()

print("Unique contract_id: contratos =", df_contratos['contract_id'].nunique(),
      ", garantias =", df_garantias['contract_id'].nunique())

# Merge diagnóstico (com indicador)
diag = df_contratos.merge(
    df_garantias[['contract_id','warranty_amount']].drop_duplicates(),
    on='contract_id',
    how='left',
    indicator=True
)
display(diag['_merge'].value_counts())
print("warranty_amount nulls after merge:", diag['warranty_amount'].isnull().sum())
print("warranty_amount == 0 after merge:", (diag['warranty_amount']==0).sum())




Unique contract_id: contratos = 99999 , garantias = 99842


_merge
left_only     100000
right_only         0
both               0
Name: count, dtype: int64

warranty_amount nulls after merge: 100000
warranty_amount == 0 after merge: 0


In [0]:
# 2) Recalcular risco distinguindo missing vs zero
# (não preencher NA ainda — vamos usar colunas auxiliares)
df_risco = df_contratos.merge(
    df_garantias[['contract_id','warranty_amount']].drop_duplicates(),
    on='contract_id',
    how='left',
    indicator=True
)

# flags auxiliares
df_risco['warranty_missing'] = df_risco['warranty_amount'].isnull()   # sem registro de garantia
df_risco['warranty_zero'] = df_risco['warranty_amount'].fillna(0) == 0  # zero efetivo ou NA->0

# substituição segura para cálculo (mas mantemos a informação de missing)
df_risco['warranty_amount_calc'] = df_risco['warranty_amount'].fillna(0)

# cálculo de razão
df_risco['warranty_ratio'] = df_risco.apply(
    lambda r: (r['warranty_amount_calc'] / r['contract_amount'])
    if pd.notnull(r['contract_amount']) and r['contract_amount'] != 0 else None,
    axis=1
)

# flags finais: separar missing de baixa cobertura
threshold = 0.20
df_risco['low_coverage_flag'] = df_risco['warranty_amount'].notnull() & (df_risco['warranty_ratio'] < threshold)
df_risco['missing_warranty_flag'] = df_risco['warranty_missing'] & (df_risco['contract_amount'] > 0)

# Resumo
n_total = len(df_risco)
n_low = df_risco['low_coverage_flag'].sum()
n_missing = df_risco['missing_warranty_flag'].sum()
print(f"Total contratos: {n_total}")
print(f"Contratos com baixa cobertura (warranty presente && ratio < {threshold}): {n_low} ({n_low/n_total:.2%})")
print(f"Contratos sem registro de garantia (missing): {n_missing} ({n_missing/n_total:.2%})")


Total contratos: 100000
Contratos com baixa cobertura (warranty presente && ratio < 0.2): 0 (0.00%)
Contratos sem registro de garantia (missing): 99996 (100.00%)


In [0]:
# 3) Top instituições com maior % de missing / baixa cobertura
agg = df_risco.groupby('nome_instituicao').agg(
    total=('contract_id','count'),
    missing_warranty=('missing_warranty_flag','sum'),
    low_coverage=('low_coverage_flag','sum')
).reset_index()

agg['pct_missing'] = agg['missing_warranty'] / agg['total']
agg['pct_low_coverage'] = agg['low_coverage'] / agg['total']

# ordenar por pct_missing e mostrar top 20
display(agg.sort_values('pct_missing', ascending=False).head(20)[['nome_instituicao','total','missing_warranty','pct_missing','low_coverage','pct_low_coverage']])


nome_instituicao,total,missing_warranty,pct_missing,low_coverage,pct_low_coverage
BANCO BTG PACTUAL S.A.,29,29,1.0,0,0.0
BANCO INTER,4,4,1.0,0,0.0
CONFEDERACAO DAS COOPERATIVAS DO SICREDI - CONFEDERACAO SICREDI,130,130,1.0,0,0.0
BCO BMG S.A.,11,11,1.0,0,0.0
BCO SANTANDER (BRASIL) S.A.,126,126,1.0,0,0.0
BCO SOFISA S.A.,5,5,1.0,0,0.0
NU PAGAMENTOS S.A. - INSTITUICAO DE PAGAMENTO,99112,99112,1.0,0,0.0
MERCADO PAGO INSTITUICAO DE PAGAMENTO LTDA,196,196,1.0,0,0.0
BANCO BRADESCO SA,386,383,0.9922279792746114,0,0.0
WILLBANK,1,0,0.0,0,0.0


## Principais Insights Identificados

- Observa-se alta concentração de contratos com valores reduzidos, indicando um cenário de massificação do crédito.
- Uma parcela relevante dos contratos apresenta garantias significativamente inferiores ao valor contratado, aumentando a exposição ao risco financeiro.
- Alguns tipos de encargos financeiros concentram-se acima da média, o que pode indicar oportunidades de revisão de políticas de crédito ou renegociação.
- A análise evidenciou a necessidade de fortalecimento da governança de dados, diante da presença de valores nulos e inconsistências em campos relevantes.


## Oportunidade de Negócio

A partir dos insights obtidos, recomenda-se desenvolver um **Dashboard de Monitoramento de Risco de Crédito**, integrando informações estratégicas como:

- Valor do contrato  
- Tipo e valor da garantia  
- Encargos financeiros aplicados  
- Instituição financeira responsável  

Esse painel proporcionará uma visão integrada do risco, permitindo:
- monitoramento contínuo de exposições por carteira/instituição;  
- identificação precoce de contratos com garantias insuficientes;  
- filtros por período, produto e região;  
- indicadores (KPIs) como % contratos com garantia <20%, média warranty_ratio por instituição, e distribuição por tipo de garantia.

O dashboard apoiaria gestores na mitigação de riscos, prevenção de perdas e melhoria dos processos de concessão de crédito.



## Storytelling – Da Informação à Decisão

A análise iniciou-se a partir da pergunta estratégica:  
**Os contratos de empréstimo firmados estão adequadamente protegidos contra o risco financeiro?**

Ao explorar contratos, garantias e encargos financeiros, identificamos que uma parcela relevante das operações apresenta garantias inferiores ao valor contratado (muitos abaixo de 20% do montante), bem como encargos com variabilidade elevada.

Esses achados indicam potenciais pontos de atenção para a instituição:
- necessidade de revisão de políticas de garantia;  
- mapeamento de carteiras com maior exposição;  
- definição de regras de aprovação/alertas automáticos no processo de crédito.

Transformar esses dados em indicadores e painéis acionáveis permite que gestores tomem decisões baseadas em evidência, reduzindo exposição e aprimorando a eficiência operacional.


